<a href="https://colab.research.google.com/github/sindgisrishti/Rebase_repo/blob/main/colab_granite_demo_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Thu Nov  6 01:14:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!python -m pip install --upgrade pip
!pip install transformers accelerate bitsandbytes sentencepiece safetensors gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 58.9 MB/s  0:00:01


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL = "ibm-granite/granite-3.2-2b-instruct"  # keep 2B version (GPU can handle it)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",             # ensures GPU placement
    torch_dtype=torch.float16,     # efficient half-precision
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("✅ Granite model loaded on GPU:", torch.cuda.get_device_name(0))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Granite model loaded on GPU: Tesla T4


In [4]:
import pandas as pd, os

os.makedirs("data", exist_ok=True)

data = [
    {"drug_a": "ibuprofen", "drug_b": "warfarin", "severity": "major",
     "description": "Increases bleeding risk due to platelet inhibition and displacement of warfarin from protein binding sites.",
     "safer_alternative": "acetaminophen"},
    {"drug_a": "amoxicillin", "drug_b": "allopurinol", "severity": "moderate",
     "description": "Concurrent use may increase risk of skin rash or hypersensitivity reactions.",
     "safer_alternative": "cephalexin"},
    {"drug_a": "lisinopril", "drug_b": "potassium chloride", "severity": "major",
     "description": "Combining may cause dangerous hyperkalemia (high potassium levels).",
     "safer_alternative": "amlodipine"},
    {"drug_a": "metformin", "drug_b": "alcohol", "severity": "moderate",
     "description": "Alcohol can increase the risk of lactic acidosis in metformin users.",
     "safer_alternative": "none"},
    {"drug_a": "paracetamol", "drug_b": "ibuprofen", "severity": "safe",
     "description": "No significant interaction. Combination commonly used for pain relief.",
     "safer_alternative": "none"},
]

pd.DataFrame(data).to_csv("data/drug_interactions.csv", index=False)
print("✅ Dataset ready.")


✅ Dataset ready.


In [12]:
# ======================================================
# 🩺 MediGuard AI — Prescription Verification System
# Enhanced Hackathon Version — Granite + Rule Engine + PDF Reports
# Author: Guided by ChatGPT (Senior AI Engineer)
# ======================================================

import gradio as gr
import pandas as pd
import json
import torch
import re
import os
from datetime import datetime
import matplotlib.pyplot as plt
import io

try:
    from fpdf import FPDF
except ImportError:
    !pip install fpdf
    from fpdf import FPDF

# ------------------------------------------------------
# ✅ Verify model pipeline exists
# ------------------------------------------------------
try:
    _ = gen("ready check", max_new_tokens=3)
    print("✅ Granite pipeline detected and ready.")
except NameError:
    raise SystemExit("❌ Load your Granite model first (see setup cell above).")

# ------------------------------------------------------
# ✅ Load Dataset
# ------------------------------------------------------
DATA_PATH = "data/drug_interactions.csv"
if not os.path.exists(DATA_PATH):
    raise SystemExit(f"❌ Dataset not found at {DATA_PATH}. Please recreate it first.")

df = pd.read_csv(DATA_PATH)
df['drug_a'] = df['drug_a'].str.lower()
df['drug_b'] = df['drug_b'].str.lower()

# ------------------------------------------------------
# Helper: JSON & regex extraction utilities
# ------------------------------------------------------

def _normalize_med_list(meds):
    out = []
    if not isinstance(meds, list): return out
    for m in meds:
        if not isinstance(m, dict): continue
        name = m.get("name") or m.get("drug") or m.get("medication") or ""
        dosage = m.get("dosage", "")
        freq = m.get("frequency", "")
        name = str(name).strip()
        if name:
            out.append({"name": name, "dosage": str(dosage).strip(), "frequency": str(freq).strip()})
    return out


def _regex_fallback_extract(text):
    """Regex-based extraction if JSON parsing fails"""
    patterns = [
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(?:\b(OD|BD|TID|QID|daily|once daily|twice daily|three times daily|prn)\b)?",
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))",
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(OD|BD|TID|QID|daily|once daily|twice daily|prn)"
    ]
    meds = []
    found = set()
    for pat in patterns:
        for match in re.finditer(pat, text, re.IGNORECASE):
            name = match.group(1).strip()
            dose = match.group(2).strip() if match.lastindex >= 2 and match.group(2) else ""
            freq = match.group(3).strip() if match.lastindex >= 3 and match.group(3) else ""
            key = (name.lower(), dose.lower(), freq.lower())
            if key not in found:
                found.add(key)
                meds.append({"name": name, "dosage": dose, "frequency": freq})
    return meds


def extract_medications(text: str):
    """LLM + Repair + Regex fallback JSON extractor (fixed version)"""
    base_prompt = f"""
    Extract structured medication information from this prescription text.
    Return ONLY valid JSON with this exact schema:
    {{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
    Text: {text}
    """
    try:
        out1 = gen(base_prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]

        # ✅ FIX: Get LAST JSON block (ignore example schema at top)
        json_matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", out1, re.DOTALL))
        if json_matches:
            json_str = json_matches[-1].group(0)
            js = json.loads(json_str)
            meds = _normalize_med_list(js.get("medications", []))
            if meds: return meds

        # Try repair if still invalid
        repair_prompt = f"Fix this to valid JSON using same schema only:\n{out1}"
        out2 = gen(repair_prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        json_matches2 = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", out2, re.DOTALL))
        if json_matches2:
            json_str2 = json_matches2[-1].group(0)
            js2 = json.loads(json_str2)
            meds = _normalize_med_list(js2.get("medications", []))
            if meds: return meds

        # Final fallback: regex extractor
        fallback = _regex_fallback_extract(text)
        if fallback:
            return fallback

        return [{"name":"Parsing error","dosage":"No valid JSON","frequency":""}]
    except Exception as e:
        return [{"name":"Parsing error","dosage":str(e),"frequency":""}]


# ------------------------------------------------------
# Drug interaction + analysis logic
# ------------------------------------------------------

def check_interactions(meds):
    interactions = []
    names = [m["name"].lower() for m in meds]
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            a, b = names[i], names[j]
            match = df[((df['drug_a']==a)&(df['drug_b']==b))|((df['drug_a']==b)&(df['drug_b']==a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": row["severity"],
                    "description": row["description"],
                    "safer_alternative": row["safer_alternative"]
                })
            else:
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": "unknown",
                    "description": "No documented interaction found.",
                    "safer_alternative": "none"
                })
    return interactions


def analyze_with_granite(age, meds, interactions):
    meds_str = ", ".join([f"{m['name']} ({m.get('dosage','')})" for m in meds])
    prompt = f"""
    You are an expert clinical pharmacist.
    A {age}-year-old patient is prescribed: {meds_str}.
    Interactions: {json.dumps(interactions, indent=2)}.
    Write a concise summary with:
    - Risk level
    - Key interactions and reasons
    - Safer alternatives
    - Final advice (2–3 lines)
    """
    try:
        out = gen(prompt, max_new_tokens=250, do_sample=False)[0]["generated_text"]
        return out.strip()
    except Exception as e:
        return f"Error generating summary: {e}"


def compute_confidence(interactions):
    base = 0.95
    penalty = 0.05 * len([i for i in interactions if i["severity"] in ["major","moderate"]])
    return round(max(0.75, base - penalty), 2)


# ------------------------------------------------------
# Visualization & PDF generator
# ------------------------------------------------------

def make_chart(interactions, path):
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for it in interactions: counts[it["severity"]] = counts.get(it["severity"],0)+1
    labels = [f"{k.capitalize()} ({v})" for k,v in counts.items() if v>0]
    sizes = [v for v in counts.values() if v>0]
    if not sizes: sizes, labels = [1], ["No Interactions"]
    plt.figure(figsize=(4,3))
    plt.pie(sizes, labels=labels, autopct="%1.0f%%")
    plt.title("Interaction Severity Distribution")
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()


def generate_pdf(report_json):
    report = json.loads(report_json)
    pdf_path = f"/tmp/mediguard_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    chart_path = pdf_path.replace(".pdf",".png")
    make_chart(report["interactions"], chart_path)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial","B",16)
    pdf.cell(0,10,"MediGuard AI - Prescription Report",ln=True,align="C")
    pdf.set_font("Arial",size=10)
    pdf.cell(0,8,f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",ln=True)
    pdf.ln(5)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Patient Details",ln=True)
    pdf.set_font("Arial",size=10)
    pdf.multi_cell(0,5,f"Age: {report['age']}")
    pdf.ln(3)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Extracted Medications",ln=True)
    pdf.set_font("Arial",size=10)
    for m in report["extracted_medications"]:
        pdf.multi_cell(0,5,f"- {m['name']} | {m['dosage']} | {m['frequency']}")
    pdf.ln(3)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Interactions",ln=True)
    pdf.set_font("Arial",size=10)
    for i in report["interactions"]:
        pdf.multi_cell(0,5,f"{i['pair']} — {i['severity'].capitalize()}\n{i['description']}\nSafer: {i['safer_alternative']}\n")

    pdf.add_page()
    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Clinical Summary",ln=True)
    pdf.set_font("Arial",size=10)
    pdf.multi_cell(0,5,report["summary_text"])
    pdf.ln(3)
    pdf.image(chart_path, x=30, w=150)
    pdf.output(pdf_path)
    return pdf_path


# ------------------------------------------------------
# Core pipeline + UI
# ------------------------------------------------------

def analyze_prescription(age, text):
    meds = extract_medications(text)
    interactions = check_interactions(meds)
    summary = analyze_with_granite(age, meds, interactions)
    confidence = compute_confidence(interactions)
    table = [[i["pair"], i["severity"], i["description"], i["safer_alternative"]] for i in interactions]
    report = {
        "project_name":"MediGuard AI",
        "age":age,
        "extracted_medications":meds,
        "interactions":interactions,
        "confidence_score":confidence,
        "summary_text":summary
    }
    return summary, table, f"{confidence*100:.1f}%", json.dumps(report, indent=2)


def app_interface(age, text):
    summary, table, confidence, report = analyze_prescription(age, text)
    disclaimer = "⚠️ For educational use only. Not a substitute for medical advice."
    return summary, table, confidence, report, disclaimer


custom_css = """
.gradio-container {font-family: 'Inter', sans-serif;}
h1 {color:#1E88E5;text-align:center;}
th {background:#1976D2;color:white;}
"""

with gr.Blocks(css=custom_css, theme="soft") as demo:
    gr.Markdown("# 🩺 MediGuard AI — Prescription Verification System")
    gr.Markdown("Granite-powered NLP + Rule-based Engine for Safe Drug Analysis")

    with gr.Row():
        age = gr.Number(label="Patient Age", value=45)
        text = gr.Textbox(label="Prescription Text", lines=4,
                          placeholder="Enter prescription or doctor's note...")

    analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    with gr.Tab("Summary & Recommendations"):
        summary_box = gr.Textbox(label="AI Summary", lines=8)
        conf_box = gr.Textbox(label="Confidence", lines=1)
        disclaimer_box = gr.Markdown()

    with gr.Tab("Detailed Interactions"):
        table = gr.Dataframe(headers=["Drug Pair","Severity","Description","Safer Alternative"])

    with gr.Tab("Downloadable Report"):
        json_box = gr.Textbox(label="Structured JSON", lines=15)
        download_btn = gr.DownloadButton(label="⬇️ Download PDF Report")

    analyze_btn.click(fn=app_interface, inputs=[age,text],
                      outputs=[summary_box, table, conf_box, json_box, disclaimer_box])



    def _pdf(json_text):
        """Generate PDF and return it as downloadable bytes for Gradio v4+."""
        pdf_path = generate_pdf(json_text)
        # Read file as bytes and send directly to Gradio
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
        return (pdf_bytes, "mediguard_report.pdf")  # (file data, filename)

    download_btn.click(fn=_pdf, inputs=[json_box], outputs=[download_btn])


demo.launch(share=True, debug=False)


✅ Granite pipeline detected and ready.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://928b451723a0ae54e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# Install OCR dependencies
!apt-get install -y tesseract-ocr
!pip install pytesseract Pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [19]:
import cv2
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance

def ocr_extract(image_path_or_array):
    """
    Enhanced OCR with preprocessing for clean text extraction.
    Works for both Gradio uploads and Colab files.
    """
    if image_path_or_array is None:
        return ""
    try:
        # Load image
        if isinstance(image_path_or_array, str) and os.path.exists(image_path_or_array):
            img = cv2.imread(image_path_or_array)
        elif isinstance(image_path_or_array, np.ndarray):
            img = cv2.cvtColor(image_path_or_array, cv2.COLOR_RGB2BGR)
        else:
            # Try to open via PIL fallback
            pil_img = Image.open(image_path_or_array)
            img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

        # --- Preprocessing pipeline ---
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Slight sharpening for clarity
        kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
        sharp = cv2.filter2D(thresh, -1, kernel)

        # Convert back to PIL for pytesseract
        processed_pil = Image.fromarray(sharp)

        # OCR with English language model
        custom_config = r"--oem 3 --psm 6 -l eng"
        text = pytesseract.image_to_string(processed_pil, config=custom_config)

        # Clean the result
        cleaned = re.sub(r"[^A-Za-z0-9\s\.,/()-]", "", text)
        cleaned = re.sub(r"\s{2,}", " ", cleaned).strip()

        if not cleaned:
            cleaned = "(No clear text detected. Try uploading a sharper image.)"

        print(f"OCR Output: {cleaned}" if DEBUG else "", end="")
        return cleaned

    except Exception as e:
        return f"OCR failed: {e}"


In [20]:
# ==========================================================
# 🩺 MediGuard AI 2.2 — Stable OCR + Granite + Dashboard + PDF
# ==========================================================

import os, re, json
from io import BytesIO
from datetime import datetime

# Core imports
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image
from fpdf import FPDF
import pytesseract

DEBUG = False  # change to True for debugging

# -----------------------------
# Check prerequisites
# -----------------------------
if "gen" not in globals():
    raise SystemExit("❌ Granite model pipeline `gen` not found. Load it first.")

DATA_PATH = "data/drug_interactions.csv"
if not os.path.exists(DATA_PATH):
    raise SystemExit("❌ data/drug_interactions.csv not found. Create it before running this.")

df = pd.read_csv(DATA_PATH)
df['drug_a'] = df['drug_a'].str.lower()
df['drug_b'] = df['drug_b'].str.lower()

# -----------------------------
# Utility functions
# -----------------------------
def safe_json_extract(text):
    """Extract last valid JSON object from model output."""
    try:
        matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", str(text), re.DOTALL))
        if not matches:
            return None
        block = matches[-1].group(0)
        return json.loads(block)
    except Exception as e:
        if DEBUG: print("safe_json_extract failed:", e)
        return None

def fallback_regex_extract(text):
    """Regex fallback if LLM JSON fails."""
    meds = []
    pattern = r"([A-Za-z][A-Za-z\-]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(\b(OD|BD|TID|QID|daily)\b)?"
    for match in re.finditer(pattern, text, re.IGNORECASE):
        name, dose, freq = match.group(1), match.group(2), match.group(3) or ""
        meds.append({"name": name, "dosage": dose, "frequency": freq})
    return meds

def extract_medications(text):
    """LLM-based JSON extraction with graceful fallback."""
    if not text.strip():
        return []
    prompt = f"""
    Extract structured medication information from this text.
    Return ONLY JSON in this format:
    {{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
    Text: {text}
    """
    try:
        out = gen(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        js = safe_json_extract(out)
        if js and "medications" in js:
            return js["medications"]
        else:
            return fallback_regex_extract(text)
    except Exception as e:
        if DEBUG: print("extract_medications failed:", e)
        return fallback_regex_extract(text)

def check_interactions(meds):
    """Match drug pairs in dataset."""
    interactions = []
    for i in range(len(meds)):
        for j in range(i+1, len(meds)):
            a, b = meds[i]["name"].lower(), meds[j]["name"].lower()
            match = df[((df["drug_a"] == a) & (df["drug_b"] == b)) |
                       ((df["drug_a"] == b) & (df["drug_b"] == a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": row["severity"],
                    "description": row["description"],
                    "safer_alternative": row["safer_alternative"]
                })
            else:
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": "unknown",
                    "description": "No documented interaction found.",
                    "safer_alternative": "-"
                })
    return interactions

def analyze_with_granite(age, meds, interactions):
    """Generate clinical summary from Granite."""
    try:
        meds_str = ", ".join([m["name"] for m in meds])
        prompt = f"""
        You are an expert clinical pharmacist.
        A {age}-year-old patient is prescribed: {meds_str}.
        Interactions: {json.dumps(interactions)}.
        Provide a concise summary:
        - Risk level
        - Key interactions and reasons
        - Safer alternatives
        - Final advice (2 lines)
        """
        result = gen(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        return result.strip()
    except Exception as e:
        if DEBUG: print("analyze_with_granite failed:", e)
        return "Error generating summary."

def make_chart(interactions):
    """Generate pie chart and return PIL image."""
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for i in interactions:
        counts[i["severity"]] = counts.get(i["severity"], 0) + 1
    labels = [f"{k} ({v})" for k,v in counts.items() if v>0]
    sizes = [v for v in counts.values() if v>0]
    colors = ["#e53935","#fb8c00","#43a047","#bdbdbd"]
    fig, ax = plt.subplots(figsize=(4,3))
    if sizes:
        ax.pie(sizes, labels=labels, autopct="%1.0f%%", colors=colors[:len(sizes)])
    else:
        ax.text(0, 0, "No interactions", ha="center", va="center")
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def compute_confidence(interactions):
    base = 0.95
    penalty = 0.05 * len([i for i in interactions if i["severity"] in ["major","moderate"]])
    return max(0.75, round(base - penalty, 2))

def generate_pdf(report):
    """Generate simple PDF report."""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "MediGuard AI - Report", ln=True, align="C")
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 8, f"Age: {report['age']}")
    pdf.multi_cell(0, 8, f"Confidence: {report['confidence_score']*100:.1f}%")
    pdf.ln(5)
    pdf.multi_cell(0, 8, "Medications:")
    for m in report["extracted_medications"]:
        pdf.multi_cell(0, 6, f"  - {m['name']} ({m['dosage']}, {m['frequency']})")
    pdf.ln(4)
    pdf.multi_cell(0, 8, "Interactions:")
    for i in report["interactions"]:
        pdf.multi_cell(0, 6, f"{i['pair']} ({i['severity']}): {i['description']}")
    pdf.ln(4)
    pdf.multi_cell(0, 8, "Summary:")
    pdf.multi_cell(0, 6, report["summary_text"])
    path = "/tmp/prescription_report.pdf"
    pdf.output(path)
    return path

def ocr_extract(image):
    if image is None: return ""
    try:
        img = Image.open(image)
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        return f"OCR error: {e}"

def analyze_prescription(age, text):
    meds = extract_medications(text)
    if not meds:
        return "No valid drugs detected.", [], "0%", None, json.dumps({})
    interactions = check_interactions(meds)
    summary = analyze_with_granite(age, meds, interactions)
    confidence = compute_confidence(interactions)
    chart = make_chart(interactions)
    table = [
        [i["pair"], i["severity"], i["description"], i["safer_alternative"]]
        for i in interactions
    ]
    report = {
        "age": age,
        "extracted_medications": meds,
        "interactions": interactions,
        "confidence_score": confidence,
        "summary_text": summary
    }
    return summary, table, f"{confidence*100:.1f}%", chart, json.dumps(report, indent=2)

def download_pdf(report_json):
    try:
        report = json.loads(report_json)
        path = generate_pdf(report)
        return path
    except Exception as e:
        return None

# -----------------------------
# Gradio Interface
# -----------------------------
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🩺 MediGuard AI 2.2 — Visual Prescription Safety Assistant")

    with gr.Row():
        img_in = gr.Image(type="filepath", label="📷 Upload Prescription Image")
        text_in = gr.Textbox(label="Extracted / Manual Prescription Text", lines=4)
        ocr_btn = gr.Button("🪄 Extract Text via OCR")

    ocr_btn.click(fn=ocr_extract, inputs=[img_in], outputs=[text_in])

    age = gr.Number(label="Patient Age", value=45)
    analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    with gr.Tab("Summary & Recommendations"):
        summary_out = gr.Textbox(label="AI Clinical Summary", lines=8)
        confidence_out = gr.Textbox(label="Confidence Score", lines=1)

    with gr.Tab("Detailed Interactions"):
        table_out = gr.Dataframe(headers=["Drug Pair", "Severity", "Description", "Safer Alternative"])

    with gr.Tab("Visualization & Report"):
        chart_out = gr.Image(label="Interaction Severity Distribution")
        json_out = gr.Textbox(label="JSON Report", lines=10)
        pdf_btn = gr.DownloadButton(label="⬇️ Download PDF Report")

    analyze_btn.click(fn=analyze_prescription,
                      inputs=[age, text_in],
                      outputs=[summary_out, table_out, confidence_out, chart_out, json_out])

    pdf_btn.click(fn=download_pdf, inputs=[json_out], outputs=[pdf_btn])

demo.launch(share=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a624c4f2f0335d9bcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
!pip install -q pytesseract Pillow fpdf rapidfuzz gradio opencv-python-headless matplotlib
!apt install -y tesseract-ocr


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [17]:
# =========================
# MediGuard AI — Stable Gradio app (fixed outputs)
# Run AFTER you have `gen` loaded and in-memory
# =========================

import os, sys, re, json, time
from io import BytesIO
from datetime import datetime

# config
DEBUG = False
DATA_DIR = "data"
DATA_PATH = os.path.join(DATA_DIR, "drug_interactions.csv")
AUDIT_LOG = os.path.join(DATA_DIR, "interaction_audit.log")
os.makedirs(DATA_DIR, exist_ok=True)

# safe imports (assume installed already in your Colab session; install if missing)
try:
    import pytesseract
    from PIL import Image
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from fpdf import FPDF
    from rapidfuzz import process, fuzz
    import gradio as gr
    import cv2
except Exception as e:
    raise SystemExit("Please install dependencies (pytesseract, Pillow, fpdf, rapidfuzz, gradio, opencv-python). Error: " + str(e))

# ensure Granite 'gen' exists
if "gen" not in globals():
    raise SystemExit("❌ Granite pipeline `gen` not found. Load the Granite pipeline as `gen` before running this cell.")

# seed dataset if missing
if not os.path.exists(DATA_PATH):
    seed = [
        {"drug_a":"amoxicillin","drug_b":"allopurinol","severity":"moderate","description":"Concurrent use may increase risk of skin rash or hypersensitivity.","safer_alternative":"cephalexin"},
        {"drug_a":"ibuprofen","drug_b":"warfarin","severity":"major","description":"NSAIDs increase bleeding risk with warfarin.","safer_alternative":"acetaminophen"}
    ]
    pd.DataFrame(seed).to_csv(DATA_PATH, index=False)

def load_df():
    df = pd.read_csv(DATA_PATH)
    df['drug_a'] = df['drug_a'].astype(str).str.lower()
    df['drug_b'] = df['drug_b'].astype(str).str.lower()
    return df

df = load_df()
canonical_drugs = sorted(set(df['drug_a'].tolist() + df['drug_b'].tolist()))

# helper functions
def audit_log(action, payload):
    entry = {"ts": datetime.now().isoformat(), "action": action, "payload": payload}
    with open(AUDIT_LOG, "a") as f:
        f.write(json.dumps(entry) + "\n")
    if DEBUG: print("AUDIT:", entry)

def snapshot_dataset():
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    snap = os.path.join(DATA_DIR, f"drug_interactions_{ts}.csv")
    pd.read_csv(DATA_PATH).to_csv(snap, index=False)
    return snap

# fuzzy match helper
def fuzzy_match_drug(name, threshold=80):
    if not name: return ""
    name_l = name.lower().strip()
    if name_l in canonical_drugs:
        return name_l
    if canonical_drugs:
        best = process.extractOne(name_l, canonical_drugs, scorer=fuzz.token_sort_ratio)
        if best and best[1] >= threshold:
            if DEBUG: print(f"Fuzzy matched {name}->{best[0]} ({best[1]})")
            return best[0]
    return name_l

# OCR preprocessing & extraction
def preprocess_for_ocr(path_or_array):
    try:
        if isinstance(path_or_array, str) and os.path.exists(path_or_array):
            arr = cv2.imread(path_or_array)
        elif isinstance(path_or_array, np.ndarray):
            arr = cv2.cvtColor(path_or_array, cv2.COLOR_RGB2BGR)
        else:
            pil = Image.open(path_or_array).convert("RGB")
            arr = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    except Exception as e:
        if DEBUG: print("OCR load error", e)
        return None
    h,w = arr.shape[:2]
    if max(h,w) > 2000:
        scale = 2000 / max(h,w)
        arr = cv2.resize(arr, (int(w*scale), int(h*scale)))
    gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray,3)
    th = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,31,15)
    kernel = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
    sharp = cv2.filter2D(th, -1, kernel)
    return Image.fromarray(sharp)

def ocr_extract(image_input):
    try:
        pil = preprocess_for_ocr(image_input)
        if pil is None:
            return ""
        text = pytesseract.image_to_string(pil, config="--oem 3 --psm 6 -l eng")
        text = re.sub(r"\n{2,}", "\n", text).strip()
        return text
    except Exception as e:
        if DEBUG: print("OCR error", e)
        return ""

# LLM extraction + regex fallback
def last_json_block(text):
    matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", str(text), re.DOTALL))
    return matches[-1].group(0) if matches else None

def regex_fallback_extract(text):
    meds=[]
    pat = r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(\b(OD|BD|TID|QID|QD|daily|once daily|twice daily|prn)\b)?"
    for m in re.finditer(pat, text, re.IGNORECASE):
        name = m.group(1).strip(); dose = m.group(2).strip(); freq = m.group(3) or ""
        meds.append({"name":name,"dosage":dose,"frequency":freq})
    return meds

def extract_medications_llm(text):
    if not text or not text.strip():
        return []
    prompt = f"""Extract ONLY JSON with schema:
{{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
Text: {text}
"""
    try:
        out = gen(prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]
        js = last_json_block(out)
        if js:
            try:
                parsed = json.loads(js)
                meds = parsed.get("medications",[])
                if meds:
                    return meds, prompt, out
            except Exception:
                # attempt repair prompt
                repair = gen(f"Repair this to valid JSON only:\n{js}", max_new_tokens=200, do_sample=False)[0]["generated_text"]
                js2 = last_json_block(repair)
                if js2:
                    parsed2 = json.loads(js2)
                    meds = parsed2.get("medications",[])
                    if meds:
                        return meds, prompt + "\n---repair---\n"+repair, out
        # fallback:
        meds = regex_fallback_extract(text)
        return meds, prompt, out
    except Exception as e:
        if DEBUG: print("LLM extraction error", e)
        meds = regex_fallback_extract(text)
        return meds, "", ""

# check interactions
def check_interactions(meds, use_fuzzy=True):
    d = load_df()
    interactions=[]
    unknown=[]
    # canonicalize names
    names=[]
    for m in meds:
        nm = m.get("name","").strip()
        nm_canon = fuzzy_match_drug(nm) if use_fuzzy else nm.lower()
        names.append(nm_canon)
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            a,b = names[i], names[j]
            match = d[((d.drug_a==a)&(d.drug_b==b))|((d.drug_a==b)&(d.drug_b==a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({"pair":f"{a} + {b}","severity":row.severity,"description":row.description,"safer_alternative":row.safer_alternative})
            else:
                interactions.append({"pair":f"{a} + {b}","severity":"unknown","description":"No documented interaction in dataset.","safer_alternative":"none"})
                unknown.append((a,b))
    return interactions, unknown

# suggest via LLM
def suggest_interaction_via_llm(a,b):
    prompt = f"""You are a clinical pharmacist. Evaluate interaction between {a} and {b}. Return JSON only: {{"severity":"major/moderate/safe/unknown","description":"one-sentence reason","safer_alternative":"drug or none"}}"""
    try:
        out = gen(prompt, max_new_tokens=120, do_sample=False)[0]["generated_text"]
        block = last_json_block(out)
        if block:
            parsed = json.loads(block)
            return parsed, prompt, out
        return {"severity":"unknown","description":out.strip(),"safer_alternative":"none"}, prompt, out
    except Exception as e:
        if DEBUG: print("suggest error", e)
        return {"severity":"unknown","description":"Suggestion failed","safer_alternative":"none"}, "", ""

# chart & pdf helpers
def make_pie_pil(interactions):
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for it in interactions:
        counts[it.get("severity","unknown")] = counts.get(it.get("severity","unknown"),0)+1
    labels = [f"{k} ({v})" for k,v in counts.items() if v>0]
    sizes = [v for v in counts.values() if v>0]
    cols = ["#e53935","#fb8c00","#43a047","#bdbdbd"]
    fig,ax = plt.subplots(figsize=(4,3))
    if sizes:
        ax.pie(sizes, labels=labels, autopct="%1.0f%%", colors=cols[:len(sizes)])
    else:
        ax.text(0.5,0.5,"No interactions",ha='center')
    buf=BytesIO(); fig.savefig(buf,format="png",bbox_inches='tight',dpi=120); plt.close(fig); buf.seek(0)
    return Image.open(buf)

def generate_pdf_bytes(report):
    chart = make_pie_pil(report.get("interactions",[]))
    tmp = "/tmp/mediguard_chart.png"; chart.save(tmp)
    pdf = FPDF(); pdf.add_page(); pdf.set_font("Arial","B",14); pdf.cell(0,10,"MediGuard AI Report",ln=True,align="C")
    pdf.ln(4); pdf.set_font("Arial",size=10); pdf.multi_cell(0,6,f"Age: {report.get('age')}")
    pdf.ln(3); pdf.set_font("Arial","B",12); pdf.cell(0,6,"Medications",ln=True); pdf.set_font("Arial",size=10)
    for m in report.get("extracted_medications",[]): pdf.multi_cell(0,6,f"- {m.get('name')} | {m.get('dosage')} | {m.get('frequency')}")
    pdf.ln(4); pdf.cell(0,6,"Interactions",ln=True); pdf.set_font("Arial",size=10)
    for it in report.get("interactions",[]): pdf.multi_cell(0,6,f"{it.get('pair')} ({it.get('severity')}) - {it.get('description')}\nAlternative: {it.get('safer_alternative')}")
    pdf.add_page(); pdf.set_font("Arial","B",12); pdf.cell(0,6,"AI Summary",ln=True); pdf.set_font("Arial",size=10); pdf.multi_cell(0,6, report.get("summary_text",""))
    try: pdf.image(tmp, x=25, w=160)
    except: pass
    out = BytesIO(); pdf.output(out); return out.getvalue()

# pipeline for UI
def analyze_pipeline(age, text, use_fuzzy=True, suggest_llm=False):
    text = (text or "").strip()
    if not text:
        return "No prescription text provided.", [], "0.0%", None, "", {}, []
    meds, extract_prompt, extract_raw = extract_medications_llm(text)
    # normalize med dicts
    meds_norm = [{"name":m.get("name",""), "dosage":m.get("dosage",""), "frequency":m.get("frequency","")} for m in meds]
    interactions, unknown_pairs = check_interactions(meds_norm, use_fuzzy=use_fuzzy)
    suggestions = []
    suggest_prompts=[]; suggest_raws=[]
    if suggest_llm and unknown_pairs:
        for a,b in unknown_pairs:
            sug, sp, sr = suggest_interaction_via_llm(a,b)
            suggestions.append([sug.get("severity","unknown"), sug.get("description",""), sug.get("safer_alternative","none")])
            suggest_prompts.append(sp); suggest_raws.append(sr)
    # summary
    try:
        meds_for_summary = [{"name":m["name"], "dosage":m.get("dosage","")} for m in meds_norm]
        summary_prompt = f"You are an expert clinical pharmacist. A {age}-year-old patient prescribed: {', '.join([m['name'] for m in meds_for_summary])}. Interactions: {json.dumps(interactions)}. Write concise summary: risk, interactions, alternatives, final advice."
        summary = gen(summary_prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
    except Exception as e:
        if DEBUG: print("Summary gen error", e)
        summary = "Error generating summary."
    confidence = max(0.75, 0.95 - 0.05*len([i for i in interactions if i.get("severity") in ("major","moderate")]))
    chart_img = None
    try:
        chart_img = make_pie_pil(interactions)
    except Exception as e:
        if DEBUG: print("Chart error", e)
        chart_img = None
    report = {"project_name":"MediGuard AI","age":age,"extracted_medications":meds_norm,"interactions":interactions,"confidence_score":confidence,"summary_text":summary}
    json_report = json.dumps(report, indent=2)
    explain = {"extract_prompt": extract_prompt, "extract_raw": extract_raw, "suggest_prompts": suggest_prompts, "suggest_raws": suggest_raws}
    # table must be list of lists for Dataframe
    table_data = [[it["pair"], it["severity"], it["description"], it["safer_alternative"]] for it in interactions]
    return summary, table_data, f"{confidence*100:.1f}%", chart_img, json_report, explain, suggestions

# KB management helpers
def add_interaction_ui(drug_a, drug_b, severity, description, safer_alt):
    drug_a = (drug_a or "").strip().lower()
    drug_b = (drug_b or "").strip().lower()
    if not drug_a or not drug_b:
        return "Provide both drug names."
    if severity in ("major","moderate") and not description.strip():
        return "Description required for major/moderate."
    df_local = load_df()
    duplicate = df_local[((df_local.drug_a==drug_a)&(df_local.drug_b==drug_b))|((df_local.drug_a==drug_b)&(df_local.drug_b==drug_a))]
    if not duplicate.empty:
        return f"Record already exists for {drug_a} + {drug_b}."
    row = {"drug_a":drug_a,"drug_b":drug_b,"severity":severity,"description":description,"safer_alternative":safer_alt}
    df_new = pd.concat([df_local, pd.DataFrame([row])], ignore_index=True)
    df_new.to_csv(DATA_PATH, index=False)
    audit_log("add", row)
    # refresh canonical list
    global canonical_drugs; canonical_drugs = sorted(set(pd.read_csv(DATA_PATH)['drug_a'].astype(str).tolist() + pd.read_csv(DATA_PATH)['drug_b'].astype(str).tolist()))
    return f"✅ Added {drug_a} + {drug_b}."

def delete_interaction_ui(drug_a, drug_b):
    da = (drug_a or "").strip().lower(); db = (drug_b or "").strip().lower()
    df_local = load_df()
    before = len(df_local)
    df_new = df_local[~(((df_local.drug_a==da)&(df_local.drug_b==db))|((df_local.drug_a==db)&(df_local.drug_b==da)))]
    if len(df_new)==before:
        return "No matching record found."
    df_new.to_csv(DATA_PATH, index=False)
    audit_log("delete", {"drug_a":da,"drug_b":db})
    global canonical_drugs; canonical_drugs = sorted(set(pd.read_csv(DATA_PATH)['drug_a'].astype(str).tolist() + pd.read_csv(DATA_PATH)['drug_b'].astype(str).tolist()))
    return f"✅ Deleted {da} + {db}."

def download_dataset_bytes():
    return (open(DATA_PATH,"rb").read(), "drug_interactions.csv")

def download_pdf_from_json(json_text):
    try:
        report = json.loads(json_text)
    except:
        report = {"summary_text": json_text}
    pdf_bytes = generate_pdf_bytes(report)
    return (pdf_bytes, "mediguard_report.pdf")

def download_ehr_from_json(json_text):
    try:
        report = json.loads(json_text)
    except:
        report = {"summary_text": json_text}
    ehr = {"patient":{"age":report.get("age")},"medications":report.get("extracted_medications",[]),"interactions":report.get("interactions",[]),"summary":report.get("summary_text")}
    return (json.dumps(ehr, indent=2).encode("utf-8"), "mediguard_ehr.json")

# Build Gradio UI (careful mapping of outputs -> component types)
custom_css = """
.gradio-container {font-family: 'Inter', sans-serif;}
h1 {color:#1E88E5;text-align:center;}
"""

with gr.Blocks(css=custom_css, title="MediGuard AI — Stable") as demo:
    gr.Markdown("# 🩺 MediGuard AI — Stable Hackathon Edition")
    with gr.Row():
        img_in = gr.Image(type="filepath", label="Upload Prescription Image (optional)")
        text_in = gr.Textbox(label="Extracted / Manual Prescription Text", lines=4)
        ocr_btn = gr.Button("🪄 Extract Text via OCR")
    ocr_btn.click(fn=ocr_extract, inputs=[img_in], outputs=[text_in])

    with gr.Row():
        age_in = gr.Number(label="Patient Age", value=45)
        fuzzy_toggle = gr.Checkbox(label="Use fuzzy drug matching", value=True)
        suggest_toggle = gr.Checkbox(label="Auto-suggest unknown pairs via Granite", value=False)
        analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    # outputs
    with gr.Tab("Summary & Recommendations"):
        summary_out = gr.Textbox(label="AI Clinical Summary", lines=8)
        conf_out = gr.Textbox(label="Confidence", lines=1)
        explain_md = gr.Markdown()
        explain_raw = gr.Textbox(label="Model Prompts & Raw Outputs (for judges)", lines=6)

    with gr.Tab("Detailed Interactions"):
        table_out = gr.Dataframe(headers=["Drug Pair","Severity","Description","Safer Alternative"])
        chart_out = gr.Image(label="Risk Distribution Chart", type="pil")

    with gr.Tab("Interactive Knowledge Base"):
        gr.Markdown("Add/Delete records")
        da = gr.Textbox(label="Drug A")
        db = gr.Textbox(label="Drug B")
        sev = gr.Dropdown(choices=["major","moderate","safe","unknown"], label="Severity", value="moderate")
        alt = gr.Textbox(label="Safer Alternative")
        desc = gr.Textbox(label="Description", lines=3)
        add_btn = gr.Button("➕ Add Interaction")
        del_btn = gr.Button("🗑 Delete Interaction")
        kb_status = gr.Markdown()
        download_csv_btn = gr.DownloadButton("⬇️ Download Dataset")

    with gr.Tab("Suggest & Review"):
        suggestions_box = gr.Dataframe(headers=["severity","description","safer_alternative"])
        accept_a = gr.Textbox(label="Drug A (to accept)")
        accept_b = gr.Textbox(label="Drug B (to accept)")
        accept_sev = gr.Dropdown(choices=["major","moderate","safe","unknown"], label="Severity")
        accept_desc = gr.Textbox(label="Description", lines=3)
        accept_alt = gr.Textbox(label="Safer alternative")
        accept_add_btn = gr.Button("✔️ Add Suggested")

    with gr.Tab("Export"):
        json_out = gr.Textbox(label="Structured JSON Report", lines=10)
        download_pdf_btn = gr.DownloadButton("⬇️ Download PDF")
        download_ehr_btn = gr.DownloadButton("⬇️ Download EHR JSON")

    # wire analyze -> outputs (must match types exactly)
    def analyze_click(age, text, use_fuzzy, suggest_llm):
        summary, table, conf_str, chart_pil, json_report, explain, suggestions = analyze_pipeline(age, text, use_fuzzy, suggest_llm)
        # format explain short and raw
        short_md = "### Explainability\n- Use the raw outputs tab to view prompts and model raw output."
        raw_text = f"EXTRACT PROMPT:\n{explain.get('extract_prompt','')}\n\nEXTRACT RAW:\n{explain.get('extract_raw','')}\n\nSUGGEST PROMPTS:\n" + "\n".join(explain.get('suggest_prompts',[])) + "\n\nSUGGEST RAWS:\n" + "\n".join(explain.get('suggest_raws',[]))
        # suggestions must be list-of-lists for Dataframe
        suggestions_rows = suggestions if isinstance(suggestions, list) else []
        return summary, table, conf_str, chart_pil, json_report, short_md, raw_text, suggestions_rows

    analyze_btn.click(fn=analyze_click, inputs=[age_in, text_in, fuzzy_toggle, suggest_toggle],
                      outputs=[summary_out, table_out, conf_out, chart_out, json_out, explain_md, explain_raw, suggestions_box])

    # KB actions
    add_btn.click(fn=lambda a,b,s,d,al: add_interaction_ui(a,b,s,d,al), inputs=[da, db, sev, desc, alt], outputs=[kb_status])
    del_btn.click(fn=lambda a,b: delete_interaction_ui(a,b), inputs=[da, db], outputs=[kb_status])
    download_csv_btn.click(fn=download_dataset_bytes, inputs=[], outputs=[download_csv_btn])

    # accept suggested -> add
    accept_add_btn.click(fn=lambda a,b,sd,dd,ad: add_interaction_ui(a,b,sd,dd,ad),
                         inputs=[accept_a, accept_b, accept_sev, accept_desc, accept_alt],
                         outputs=[kb_status])

    # exports
    download_pdf_btn.click(fn=download_pdf_from_json, inputs=[json_out], outputs=[download_pdf_btn])
    download_ehr_btn.click(fn=download_ehr_from_json, inputs=[json_out], outputs=[download_ehr_btn])

    import logging, warnings, asyncio
logging.getLogger("uvicorn.error").setLevel(logging.CRITICAL)
logging.getLogger("uvicorn.access").setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore", category=RuntimeWarning)
asyncio.set_event_loop(asyncio.new_event_loop())

demo.launch(share=True, debug=DEBUG)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://268c0e0dcafc060f70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
# MediGuard AI — Final Stable WOW Edition (single cell)
# Run this AFTER you have `gen` in memory (your Granite pipeline)

import os, re, json, traceback, asyncio, warnings, logging
from io import BytesIO
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from PIL import Image
from fpdf import FPDF

# try safe imports, install if missing (Colab-friendly)
try:
    import pytesseract, cv2, qrcode
    from rapidfuzz import process, fuzz
    import gradio as gr
    from gtts import gTTS
except Exception:
    print("Installing missing packages (this may take ~60s)...")
    # package installs
    !pip install -q pytesseract Pillow fpdf rapidfuzz gradio opencv-python-headless matplotlib gTTS qrcode pandas
    !apt-get install -y -qq tesseract-ocr
    import pytesseract, cv2, qrcode
    from rapidfuzz import process, fuzz
    import gradio as gr
    from gtts import gTTS

# CONFIG
DEBUG = False
DATA_DIR = "data"
DATA_PATH = os.path.join(DATA_DIR, "drug_interactions.csv")
os.makedirs(DATA_DIR, exist_ok=True)

# require user loaded gen
if "gen" not in globals():
    raise SystemExit("❌ Granite pipeline `gen` not found. Load your model pipeline as `gen` before running this cell.")

# seed dataset if missing
if not os.path.exists(DATA_PATH):
    seed = [
        {"drug_a":"amoxicillin","drug_b":"allopurinol","severity":"moderate","description":"Concurrent use may increase risk of skin rash or hypersensitivity reactions.","safer_alternative":"cephalexin"},
        {"drug_a":"ibuprofen","drug_b":"warfarin","severity":"major","description":"NSAIDs increase bleeding risk with warfarin.","safer_alternative":"acetaminophen"},
        {"drug_a":"lisinopril","drug_b":"spironolactone","severity":"major","description":"Increased risk of hyperkalemia when combined.","safer_alternative":"thiazide diuretic"},
    ]
    pd.DataFrame(seed).to_csv(DATA_PATH, index=False)

# ---------- helpers ----------
def load_df():
    df = pd.read_csv(DATA_PATH)
    df['drug_a'] = df['drug_a'].astype(str).str.lower()
    df['drug_b'] = df['drug_b'].astype(str).str.lower()
    return df

def build_canonical():
    df = load_df()
    names = set(df['drug_a'].tolist() + df['drug_b'].tolist())
    return sorted([n for n in names if isinstance(n, str) and n.strip()])

canonical_drugs = build_canonical()

def fuzzy_match_drug(name, threshold=80):
    if not name: return ""
    name_l = name.lower().strip()
    if name_l in canonical_drugs:
        return name_l
    if canonical_drugs:
        best = process.extractOne(name_l, canonical_drugs, scorer=fuzz.token_sort_ratio)
        if best and best[1] >= threshold:
            return best[0]
    return name_l

# ---------- OCR ----------
def preprocess_for_ocr(path_or_array):
    try:
        if isinstance(path_or_array, str) and os.path.exists(path_or_array):
            arr = cv2.imread(path_or_array)
        elif isinstance(path_or_array, np.ndarray):
            arr = cv2.cvtColor(path_or_array, cv2.COLOR_RGB2BGR)
        else:
            pil = Image.open(path_or_array).convert("RGB")
            arr = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    except Exception as e:
        if DEBUG: print("OCR load error", e)
        return None
    h,w = arr.shape[:2]
    if max(h,w) > 2000:
        scale = 2000 / max(h,w)
        arr = cv2.resize(arr, (int(w*scale), int(h*scale)))
    gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 3)
    th = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,31,15)
    return Image.fromarray(th)

def ocr_extract(image_input):
    if not image_input:
        return ""
    pil = preprocess_for_ocr(image_input)
    if pil is None:
        return ""
    text = pytesseract.image_to_string(pil, config="--oem 3 --psm 6 -l eng")
    text = re.sub(r"\n{2,}", "\n", text).strip()
    if DEBUG: print("OCR text:", text)
    return text

# ---------- LLM extraction and regex fallback ----------
def last_json_block(text):
    matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", str(text), re.DOTALL))
    return matches[-1].group(0) if matches else None

def regex_fallback_extract(text):
    meds=[]
    pat = r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(\b(OD|BD|TID|QID|QD|daily|once daily|twice daily|prn)\b)?"
    for m in re.finditer(pat, text, re.IGNORECASE):
        name = m.group(1).strip(); dose = m.group(2).strip(); freq = m.group(3) or ""
        meds.append({"name":name,"dosage":dose,"frequency":freq})
    return meds

def extract_medications_llm(text):
    if not text or not text.strip():
        return [], "", ""
    # instruct strict json only
    prompt = f"""Extract ONLY JSON with schema:
{{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
Text: {text}
"""
    try:
        out = gen(prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]
        js = last_json_block(out)
        if js:
            try:
                parsed = json.loads(js)
                meds = parsed.get("medications",[])
                if meds:
                    return meds, prompt, out
            except Exception as e:
                if DEBUG: print("JSON parse error:", e)
                # attempt to repair using LLM
                try:
                    repair = gen(f"Repair this to valid JSON only:\n{js}", max_new_tokens=200, do_sample=False)[0]["generated_text"]
                    js2 = last_json_block(repair)
                    if js2:
                        parsed2 = json.loads(js2)
                        meds = parsed2.get("medications",[])
                        if meds:
                            return meds, prompt + "\n---repair---\n" + repair, out
                except Exception as e2:
                    if DEBUG: print("Repair failed:", e2)
        # fallback regex
        meds = regex_fallback_extract(text)
        return meds, prompt, out
    except Exception as e:
        if DEBUG: print("LLM extraction exception:", e)
        meds = regex_fallback_extract(text)
        return meds, "", ""

# ---------- rule-based interaction check & LLM suggest ----------
def check_interactions(meds, use_fuzzy=True):
    d = load_df()
    interactions=[]
    unknown=[]
    names=[]
    for m in meds:
        nm = m.get("name","").strip()
        nm_canon = fuzzy_match_drug(nm) if use_fuzzy else nm.lower()
        names.append(nm_canon)
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            a,b = names[i], names[j]
            match = d[((d.drug_a == a) & (d.drug_b == b)) | ((d.drug_a == b) & (d.drug_b == a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({"pair":f"{a} + {b}", "severity":row["severity"], "description":row["description"], "safer_alternative":row["safer_alternative"]})
            else:
                interactions.append({"pair":f"{a} + {b}", "severity":"unknown", "description":"No documented interaction in dataset.", "safer_alternative":"none"})
                unknown.append((a,b))
    return interactions, unknown

def suggest_interaction_via_llm(a,b):
    prompt = f"""You are a clinical pharmacist. Evaluate interaction between {a} and {b}. Return JSON only: {{'severity':'major/moderate/safe/unknown','description':'one-sentence reason','safer_alternative':'drug or none'}}"""
    try:
        out = gen(prompt, max_new_tokens=120, do_sample=False)[0]["generated_text"]
        block = last_json_block(out)
        if block:
            parsed = json.loads(block.replace("'", '"'))
            return parsed, prompt, out
        return {"severity":"unknown","description":out.strip(),"safer_alternative":"none"}, prompt, out
    except Exception as e:
        if DEBUG: print("suggest LLM error", e)
        return {"severity":"unknown","description":"Suggestion failed","safer_alternative":"none"}, "", ""

# ---------- visuals, QR, audio, PDF ----------
def make_pie_pil(interactions):
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for it in interactions:
        counts[it.get("severity","unknown")] = counts.get(it.get("severity","unknown"),0)+1
    labels = [f"{k} ({v})" for k,v in counts.items()]
    sizes = list(counts.values())
    fig,ax = plt.subplots(figsize=(5,4))
    ax.pie(sizes, labels=labels, autopct="%1.0f%%", textprops={'fontsize':8})
    buf=BytesIO(); fig.savefig(buf,format="png",bbox_inches='tight',dpi=120); plt.close(fig); buf.seek(0)
    return Image.open(buf)

def make_heatmap_pil(interactions):
    df = load_df()
    drugs = sorted(list(set(df['drug_a'].tolist() + df['drug_b'].tolist())))
    if len(drugs) > 20: drugs = drugs[:20]
    idx = {d:i for i,d in enumerate(drugs)}
    mat = np.zeros((len(drugs), len(drugs)))
    severity_score = {"major":2, "moderate":1, "safe":0, "unknown":0}
    for it in interactions:
        a,b = [x.strip() for x in it['pair'].split('+')]
        if a in idx and b in idx:
            s = severity_score.get(it.get("severity","unknown"), 0)
            mat[idx[a], idx[b]] = max(mat[idx[a], idx[b]], s)
            mat[idx[b], idx[a]] = max(mat[idx[b], idx[a]], s)
    fig,ax = plt.subplots(figsize=(6,5))
    cax = ax.imshow(mat, cmap="Reds", interpolation='nearest')
    ax.set_xticks(range(len(drugs))); ax.set_xticklabels(drugs, rotation=90, fontsize=7)
    ax.set_yticks(range(len(drugs))); ax.set_yticklabels(drugs, fontsize=7)
    fig.colorbar(cax, ax=ax, shrink=0.6)
    buf = BytesIO(); fig.savefig(buf, format="png", bbox_inches='tight', dpi=120); plt.close(fig); buf.seek(0)
    return Image.open(buf)

def make_qr_bytes(json_report):
    qr = qrcode.QRCode(box_size=4, border=2)
    qr.add_data(json_report)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    buf = BytesIO(); img.save(buf, format="PNG"); buf.seek(0)
    return buf.getvalue()

def make_audio_bytes(summary_text, lang="en"):
    try:
        tts = gTTS(text=summary_text or "No summary", lang=lang)
        buf = BytesIO(); tts.write_to_fp(buf); buf.seek(0)
        return buf.getvalue()
    except Exception as e:
        if DEBUG: print("gTTS error:", e)
        return b""

def generate_pdf_bytes(report):
    # simple PDF with summary and interactions + pie chart
    chart = make_pie_pil(report.get("interactions", []))
    tmp = "/tmp/mediguard_chart.png"; chart.save(tmp)
    pdf = FPDF(); pdf.set_auto_page_break(auto=True, margin=12)
    pdf.add_page(); pdf.set_font("Arial","B",14); pdf.cell(0,8,"MediGuard AI — Prescription Report", ln=True, align="C")
    pdf.ln(4); pdf.set_font("Arial",size=10); pdf.cell(0,6,f"Generated: {datetime.now().isoformat()}", ln=True)
    pdf.ln(4); pdf.set_font("Arial","B",12); pdf.cell(0,6,"Medications & Interactions", ln=True); pdf.set_font("Arial",size=10)
    for m in report.get("extracted_medications", []):
        pdf.multi_cell(0,6, f"- {m.get('name','')} | {m.get('dosage','')} | {m.get('frequency','')}")
    pdf.ln(3)
    for it in report.get("interactions", []):
        pdf.multi_cell(0,6, f"{it.get('pair')} ({it.get('severity')}) - {it.get('description')}\nAlternative: {it.get('safer_alternative')}\n")
    pdf.add_page(); pdf.set_font("Arial","B",12); pdf.cell(0,6,"AI Clinical Summary", ln=True); pdf.set_font("Arial", size=10)
    pdf.multi_cell(0,6, report.get("summary_text",""))
    try: pdf.image(tmp, x=25, w=160)
    except: pass
    out = BytesIO(); pdf.output(out); return out.getvalue()

# ---------- pipeline ----------
def analyze_pipeline(age, text, use_fuzzy=True, suggest_llm=False, correct_spell=True):
    text = (text or "").strip()
    if not text:
        return "No prescription text provided.", [], "0.0%", None, None, "{}", {"extract_prompt":"", "extract_raw":""}, []
    meds, extract_prompt, extract_raw = extract_medications_llm(text)
    meds_norm = [{"name": m.get("name","").strip(), "dosage": m.get("dosage",""), "frequency": m.get("frequency","")} for m in meds]
    if correct_spell:
        for m in meds_norm:
            corrected = fuzzy_match_drug(m['name'])
            if corrected and corrected != m['name'].lower():
                m['name'] = corrected
    interactions, unknown_pairs = check_interactions(meds_norm, use_fuzzy=use_fuzzy)
    suggestions = []
    suggest_prompts=[]; suggest_raws=[]
    if suggest_llm and unknown_pairs:
        for a,b in unknown_pairs:
            sug, sp, sr = suggest_interaction_via_llm(a,b)
            suggestions.append([sug.get("severity","unknown"), sug.get("description",""), sug.get("safer_alternative","none")])
            suggest_prompts.append(sp); suggest_raws.append(sr)
    # summary via Granite (defensive)
    try:
        meds_str = ", ".join([f"{m['name']} ({m.get('dosage','')})" for m in meds_norm]) or "No medications"
        summ_prompt = f"You are an expert clinical pharmacist. A {age}-year-old patient is prescribed: {meds_str}. Interactions: {json.dumps(interactions)}. Write a concise clinical summary (risk level, notable interactions & reasons, safer alternatives, final advice). Plain text only."
        summary = gen(summ_prompt, max_new_tokens=220, do_sample=False)[0]["generated_text"].strip()
    except Exception as e:
        if DEBUG: print("Summary gen error:", e)
        summary = "Error generating summary."
    confidence = max(0.75, 0.95 - 0.05 * len([i for i in interactions if i.get("severity") in ("major","moderate")]))
    chart_img = None; heatmap_img = None
    try:
        chart_img = make_pie_pil(interactions)
        heatmap_img = make_heatmap_pil(interactions)
    except Exception as e:
        if DEBUG: print("Chart/heatmap error:", e)
    report = {"project_name":"MediGuard AI", "age":age, "extracted_medications":meds_norm, "interactions":interactions, "confidence_score":confidence, "summary_text":summary}
    json_report = json.dumps(report, indent=2)
    explain = {"extract_prompt": extract_prompt, "extract_raw": extract_raw, "suggest_prompts": suggest_prompts, "suggest_raws": suggest_raws}
    table_data = [[it["pair"], it["severity"], it["description"], it["safer_alternative"]] for it in interactions]
    return summary, table_data, f"{confidence*100:.1f}%", chart_img, heatmap_img, json_report, explain, suggestions

# ---------- KB helpers ----------
def add_interaction_ui(drug_a, drug_b, severity, description, safer_alt):
    a = (drug_a or "").strip().lower(); b = (drug_b or "").strip().lower()
    if not a or not b: return "Provide both drugs."
    df_local = load_df()
    dup = df_local[((df_local.drug_a==a)&(df_local.drug_b==b))|((df_local.drug_a==b)&(df_local.drug_b==a))]
    if not dup.empty: return f"Record exists for {a} + {b}."
    row = {"drug_a":a,"drug_b":b,"severity":severity,"description":description or "", "safer_alternative":safer_alt or ""}
    df_new = pd.concat([df_local, pd.DataFrame([row])], ignore_index=True)
    df_new.to_csv(DATA_PATH, index=False)
    global canonical_drugs; canonical_drugs = build_canonical()
    return f"✅ Added {a} + {b}."

def delete_interaction_ui(drug_a, drug_b):
    a = (drug_a or "").strip().lower(); b = (drug_b or "").strip().lower()
    df_local = load_df(); before = len(df_local)
    df_new = df_local[~(((df_local.drug_a==a)&(df_local.drug_b==b))|((df_local.drug_a==b)&(df_local.drug_b==a)))]
    if len(df_new)==before: return "No matching record."
    df_new.to_csv(DATA_PATH, index=False); global canonical_drugs; canonical_drugs = build_canonical()
    return f"✅ Deleted {a} + {b}."

def download_dataset_bytes():
    return (open(DATA_PATH,"rb").read(), "drug_interactions.csv")

def download_pdf_from_json(json_text):
    try:
        report = json.loads(json_text)
    except:
        report = {"summary_text": json_text}
    return (generate_pdf_bytes(report), "mediguard_report.pdf")

def download_ehr_from_json(json_text):
    try:
        report = json.loads(json_text)
    except:
        report = {"summary_text": json_text}
    ehr = {"patient":{"age": report.get("age")}, "medications": report.get("extracted_medications", []), "interactions": report.get("interactions", []), "summary": report.get("summary_text","")}
    return (json.dumps(ehr, indent=2).encode("utf-8"), "mediguard_ehr.json")

def download_qr_from_json(json_text):
    try:
        return (make_qr_bytes(json_text), "mediguard_qr.png")
    except:
        return (b"", "qr.png")

def download_audio_from_summary(summary_text):
    try:
        return (make_audio_bytes(summary_text), "summary.mp3")
    except:
        return (b"", "summary.mp3")

# ---------- GRADIO UI ----------
custom_css = """
.gradio-container {font-family: 'Inter', sans-serif;}
h1 {color:#1E88E5;text-align:center;}
"""

with gr.Blocks(css=custom_css, title="MediGuard AI — WOW Edition") as demo:
    gr.Markdown("# 🩺 MediGuard AI — WOW Edition")
    gr.Markdown("Multimodal prescription verifier — Granite LLM + rule-engine + editable KB + explainability + dashboard + export")

    with gr.Row():
        img_in = gr.Image(type="filepath", label="Upload Prescription Image (optional)")
        text_in = gr.Textbox(label="Extracted / Manual Prescription Text", lines=4, placeholder="Enter or paste prescription here...")
        ocr_btn = gr.Button("🪄 Extract Text via OCR")
    ocr_btn.click(fn=ocr_extract, inputs=[img_in], outputs=[text_in])

    with gr.Row():
        age_in = gr.Number(label="Patient Age", value=45)
        fuzzy_toggle = gr.Checkbox(label="Use fuzzy drug matching", value=True)
        suggest_toggle = gr.Checkbox(label="Auto-suggest unknown pairs via Granite (preview)", value=False)
        spell_toggle = gr.Checkbox(label="Auto-correct drug names (fuzzy)", value=True)
        analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    with gr.Tab("Summary & Recommendations"):
        summary_out = gr.Textbox(label="AI Clinical Summary", lines=8)
        conf_out = gr.Textbox(label="Confidence", lines=1)
        play_audio = gr.Audio(label="🔊 AI Summary Audio")
        qr_img = gr.Image(label="📱 Summary QR Code", type="pil")
        download_audio_btn = gr.DownloadButton("⬇️ Download Audio")
        download_qr_btn = gr.DownloadButton("⬇️ Download QR")

    with gr.Tab("Detailed Interactions"):
        table_out = gr.Dataframe(headers=["Drug Pair","Severity","Description","Safer Alternative"])
        chart_out = gr.Image(label="Interaction Severity Distribution", type="pil")
        heatmap_out = gr.Image(label="Interaction Heatmap", type="pil")

    with gr.Tab("Interactive Knowledge Base"):
        gr.Markdown("Add / Delete interaction records (editable KB)")
        da = gr.Textbox(label="Drug A")
        db = gr.Textbox(label="Drug B")
        sev = gr.Dropdown(choices=["major","moderate","safe","unknown"], label="Severity", value="moderate")
        alt = gr.Textbox(label="Safer Alternative")
        desc = gr.Textbox(label="Description", lines=3)
        add_btn = gr.Button("➕ Add Interaction")
        del_btn = gr.Button("🗑 Delete Interaction")
        kb_status = gr.Markdown()
        download_csv_btn = gr.DownloadButton("⬇️ Download Dataset")

    with gr.Tab("Suggest & Review"):
        suggestions_box = gr.Dataframe(headers=["severity","description","safer_alternative"])
        accept_a = gr.Textbox(label="Drug A (to accept)")
        accept_b = gr.Textbox(label="Drug B (to accept)")
        accept_sev = gr.Dropdown(choices=["major","moderate","safe","unknown"], label="Severity")
        accept_desc = gr.Textbox(label="Description", lines=3)
        accept_alt = gr.Textbox(label="Safer alternative")
        accept_add_btn = gr.Button("✔️ Add Suggested")

    with gr.Tab("Export"):
        json_out = gr.Textbox(label="Structured JSON Report", lines=10)
        download_pdf_btn = gr.DownloadButton("⬇️ Download PDF")
        download_ehr_btn = gr.DownloadButton("⬇️ Download EHR JSON")
        download_ds_btn = gr.DownloadButton("⬇️ Download KB CSV")

    with gr.Tab("Explainability (Judges)"):
        explain_raw = gr.Textbox(label="Model Prompts & Raw Outputs", lines=12)

    # analyze button behaviour
    def analyze_click(age, text, use_fuzzy, suggest_llm, correct_spell):
        try:
            summary, table, conf_str, chart_pil, heatmap_pil, json_report, explain, suggestions = analyze_pipeline(age, text, use_fuzzy, suggest_llm, correct_spell)
            # audio and qr
            audio_bytes = make_audio_bytes(summary) if summary else b""
            qr_bytes = make_qr_bytes(json_report)
            # safe conversions for PIL outputs
            chart_safe = chart_pil if isinstance(chart_pil, Image.Image) else None
            heatmap_safe = heatmap_pil if isinstance(heatmap_pil, Image.Image) else None
            qr_img = Image.open(BytesIO(qr_bytes)) if qr_bytes else None
            # suggestions rows
            suggestions_rows = suggestions if isinstance(suggestions, list) else []
            return summary, table, conf_str, chart_safe, heatmap_safe, (audio_bytes, "summary.mp3"), qr_img, json_report, explain.get("extract_raw",""), suggestions_rows
        except Exception as e:
            print("🔥 ERROR in analyze_click:", e)
            traceback.print_exc()
            # return safe fallbacks (matching output order)
            return "Error during analysis.", [], "0%", None, None, (b"", "summary.mp3"), None, "{}", "", []

    analyze_btn.click(fn=analyze_click,
                      inputs=[age_in, text_in, fuzzy_toggle, suggest_toggle, spell_toggle],
                      outputs=[summary_out, table_out, conf_out, chart_out, heatmap_out, play_audio, qr_img, json_out, explain_raw, suggestions_box])

    # KB actions
    add_btn.click(fn=lambda a,b,s,d,al: add_interaction_ui(a,b,s,d,al), inputs=[da, db, sev, desc, alt], outputs=[kb_status])
    del_btn.click(fn=lambda a,b: delete_interaction_ui(a,b), inputs=[da, db], outputs=[kb_status])
    download_csv_btn.click(fn=download_dataset_bytes, inputs=[], outputs=[download_ds_btn])

    # accept suggested -> add
    accept_add_btn.click(fn=lambda a,b,sd,dd,ad: add_interaction_ui(a,b,sd,dd,ad),
                         inputs=[accept_a, accept_b, accept_sev, accept_desc, accept_alt],
                         outputs=[kb_status])

    # exports
    download_pdf_btn.click(fn=download_pdf_from_json, inputs=[json_out], outputs=[download_pdf_btn])
    download_ehr_btn.click(fn=download_ehr_from_json, inputs=[json_out], outputs=[download_ehr_btn])
    download_ds_btn.click(fn=download_dataset_bytes, inputs=[], outputs=[download_ds_btn])
    download_audio_btn.click(fn=download_audio_from_summary, inputs=[summary_out], outputs=[download_audio_btn])
    download_qr_btn.click(fn=download_qr_from_json, inputs=[json_out], outputs=[download_qr_btn])

    # safety: new event loop for Colab
    logging.getLogger("uvicorn.error").setLevel(logging.CRITICAL)
    logging.getLogger("uvicorn.access").setLevel(logging.CRITICAL)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    try:
        asyncio.set_event_loop(asyncio.new_event_loop())
    except Exception:
        pass

# launch
demo.launch(share=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://14d24e944e3d60552f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
